In [1]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import io
from PIL import Image

In [2]:
df = pd.read_csv('dados/public.csv')
df_ap = pd.read_csv('dados\AerodromosPublicos.csv', sep=';')
df_ap = df_ap[['Código OACI', 'LATGEOPOINT','LONGEOPOINT']]
df_ap = df_ap.rename({'LATGEOPOINT': 'LATGEOPOINT_OR','LONGEOPOINT':'LONGEOPOINT_OR'},axis=1)
df = df.merge(df_ap, left_on='origem', right_on='Código OACI',how='left')
df_ap = df_ap.rename({'LATGEOPOINT_OR': 'LATGEOPOINT_DE','LONGEOPOINT_OR':'LONGEOPOINT_DE'},axis=1)
df = df.merge(df_ap, left_on='destino', right_on='Código OACI',how='left')

<>:2: SyntaxWarning: invalid escape sequence '\A'
<>:2: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Eduardo\AppData\Local\Temp\ipykernel_13792\1634027118.py:2: SyntaxWarning: invalid escape sequence '\A'
  df_ap = pd.read_csv('dados\AerodromosPublicos.csv', sep=';')


In [3]:
df_train = df[~df.espera.isnull()]
df_test = df[df.espera.isnull()]

In [4]:
def load_crop_img(path):
    image = Image.open(path)
    w = image.width
    h = image.height
    img_brasil = image.crop((1030, 820, w-250, h-620))
    return img_brasil

In [5]:
def paint_airport(image, lat_airport, lon_airport, color = [148,0,211]):

    # Mapas de latitude e longitude do Brasil
    lat_map = np.linspace(-33.750812, 5.235216, 888)
    lon_map = np.linspace(-73.981645, -34.792202, 912)

    # Posicao do aeroporto na imagem
    lat_result = np.abs(lat_map - lat_airport).argmin()
    lon_result = np.abs(lon_map - lon_airport).argmin()

    # Ajuste da latitude
    lat_result = 888 - lat_result

    # Pinta a imagem
    image[lat_result - 25 : lat_result + 25, lon_result - 25 : lon_result + 25 , :] = color 

    return image

In [6]:
for id, url, espera, lat_or, lon_or, lat_de, lon_de in tqdm(zip(df_train.flightid ,df_train.url_img_satelite, df_train.espera, df_train.LATGEOPOINT_OR, df_train.LONGEOPOINT_OR, df_train.LATGEOPOINT_DE, df_train.LONGEOPOINT_DE)):
    if espera:
        label = 'atrasado'
    else:
        label = 'pontual'
    try:
        image_name = url.split('/')[-1]
        image = load_crop_img('imgs/' + image_name)
        np_img = np.array(image)
        img_ap = paint_airport(np_img, float(lat_or), float(lon_or))
        img_ap = paint_airport(np_img, float(lat_de), float(lon_de), [225,0,132])
        image_final = Image.fromarray(img_ap)
        image_final = image_final.resize((224,224))
        image_final.save(f'dados/train/{label}/{id}.jpg')
    except:
        continue

0it [00:00, ?it/s]

211679it [1:32:48, 38.01it/s]  


In [7]:
# for id, url, espera, lat_or, lon_or, lat_de, lon_de in tqdm(zip(df_test.flightid ,df_test.url_img_satelite, df_test.espera, df_test.LATGEOPOINT_OR, df_test.LONGEOPOINT_OR, df_test.LATGEOPOINT_DE, df_test.LONGEOPOINT_DE)):
#     try:
#         image_name = url.split('/')[-1]
#         image = load_crop_img('imgs/' + image_name)
#         np_img = np.array(image)
#         img_ap = paint_airport(np_img, float(lat_or), float(lon_or))
#         img_ap = paint_airport(np_img, float(lat_de), float(lon_de), [225,0,132])
#         image_final = Image.fromarray(img_ap)
#         image_final = image_final.resize((224,224))
#         image_final.save(f'dados/test/test/{id}.jpg')
#     except:
#         continue